In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.stats import linregress

import sys
sys.path.append('../')

from geometry import Geometry
from forcing import Forcing
from layer import LayerModel

import warnings
#warnings.filterwarnings('ignore')

np.set_printoptions(precision=2)

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
years = np.arange(1979,2013)
x1,x2 = 245.4-360,246.2-360
y1,y2 = -75.13,-74.85

N = 3
ress = ['0.5','1.0','2.0']
tpers = [30,30,30]

cols = ['teal','darkgoldenrod','firebrick']

In [ ]:
def getav(melt,xx,yy,x1,x2,y1,y2):
    mm = melt.copy()
    mm = np.where(yy<y1,np.nan,mm)
    mm = np.where(yy>y2,np.nan,mm)
    mm = np.where(xx<x1,np.nan,mm)
    mm = np.where(xx>x2,np.nan,mm)
    return np.nanmean(mm)

In [ ]:
def getav_field(melt,xx,yy,x1,x2,y1,y2):
    mm = melt.copy()
    mm = np.where(yy<y1,np.nan,mm)
    mm = np.where(yy>y2,np.nan,mm)
    mm = np.where(xx<x1,np.nan,mm)
    mm = np.where(xx>x2,np.nan,mm)
    return mm

In [ ]:
def getdisc2(ds,dx):
    x1,x2,y1,y2=-1.588e6,-1.530e6,-690000,-645000
    ds = ds.sel(x=slice(x1,x2),y=slice(y1,y2))
    melt = ds.melt

    xarr = ds.x.values
    yarr = ds.y.values    

    a = 5.45e-01
    b = 1.81e5
    bplus = 10000
    bmin = 5000

    Nvals = int((bplus+bmin)/dx)
    ymin = a*xarr+b-bmin
    
    xarr = ds.x.values
    yarr = ds.y.values
    mmax = 0*xarr
    yval = 0*xarr
    for i,ii in enumerate(xarr):
        yidx = np.argmin((yarr-ymin[i])**2)
        mmelt = melt[yidx:yidx+Nvals,i]
        yyarr = yarr[yidx:yidx+Nvals]
        mmax[i]=np.nanmax(mmelt)
        yval[i]=yyarr[np.nanargmax(mmelt)]

    return np.nanmean(mmax)

In [ ]:
def getdisc3(ds,dx):
    x1,x2,y1,y2=-1.588e6,-1.530e6,-690000,-640000
    ds = ds.sel(x=slice(x1,x2),y=slice(y1,y2))
    melt = ds.melt    
    
    xarr = ds.x.values
    yarr = ds.y.values    

    a = 5.45e-01
    b = 1.81e5
    bplus = 10000
    bmin = 5000

    Nvals = int((bplus+bmin)/dx)
    ymin = a*xarr+b-bmin

    mmelt = np.zeros((Nvals,len(xarr)))
    yval  = np.zeros((Nvals,len(xarr)))
    for i,ii in enumerate(xarr):
        yidx = np.argmin((yarr-ymin[i])**2)
        yval[:,i] = yarr[yidx:yidx+Nvals]
        mmelt[:,i] = melt[yidx:yidx+Nvals,i]

    return np.nanmean(mmelt)

In [ ]:
def getdiags(ds):
    xarr = ds.x.values
    dx = xarr[1]-xarr[0]
    
    xx,yy = np.meshgrid(ds.x,ds.y)
    melt = xr.where(ds.tmask,ds.melt,np.nan).copy()
    mcd   = np.nanmean(melt) #Average melt [m/yr]
    mkw   = getav(melt,xx,yy,-1.505e6,-1.400e6,-700000,-660000) #Kohler West melt [m/yr]
    #mdc   = getdisc(melt,dx,xx,yy) #DISC melt [m/yr]
    mdc   = getdisc3(ds,dx) #Dotson Channel melt [m/yr]
    mcc  = getdisc2(ds,dx) #Channel Center melt [m/yr]
    over = 1e-6*((ds.entr+ds.ent2)*ds.tmask*dx*dx).sum().values/(3600*24*365.25) #Total overturning [Sv]
    return mcd,mkw,mdc,mcc,over

In [ ]:
MCD = {}
MKW = {}
MDC = {}
MCC = {}
OVE = {}

for r,res in enumerate(ress):
    MCD[res] = np.nan*np.zeros(len(years))
    MKW[res] = np.nan*np.zeros(len(years))
    MDC[res] = np.nan*np.zeros(len(years))
    MCC[res] = np.nan*np.zeros(len(years))
    OVE[res] = np.nan*np.zeros(len(years))
    for y,yy in enumerate(years):
        try:
            ds = xr.open_dataset(f'../../results/CrossDots_{res}_mitgcm_{yy}_{yy}_{tpers[r]:03.0f}.nc')
            MCD[res][y],MKW[res][y],MDC[res][y],MCC[res][y],OVE[res][y] = getdiags(ds)
            ds.close()
        except:
            pass
        print(res,yy,MCD[res][y],MKW[res][y],MDC[res][y],MCC[res][y],OVE[res][y])

In [ ]:


#MITgcm

ds = xr.open_dataset('../../../data/paulholland/melt.nc')
ds = ds.sel(LONGITUDE=slice(360-114.7,360-109),LATITUDE=slice(-75.3,-74.1))#,TIME=timep)
melt = xr.where(ds.melt==0,np.nan,ds.melt)
melt = melt.mean(dim=['LONGITUDE','LATITUDE'],skipna=True)
MCD = np.zeros(len(years))
for y,yy in enumerate(years):
    MCD[y] = melt.sel(TIME=slice(f"{yy}-1-1",f"{yy}-12-31")).mean()
ds.close()

ds = xr.open_dataset('../../../data/paulholland/melt.nc')
ds = ds.sel(LONGITUDE=slice(360-114.7,360-109),LATITUDE=slice(-75.3+.05,-74.1+.05))#,TIME=timep)
lon = (ds.LONGITUDE - 360.).values
lat = (ds.LATITUDE - .05).values
llon,llat = np.meshgrid(lon,lat)
MKW = np.zeros(len(years))
for y,yy in enumerate(years):
    mmelt = ds.melt.sel(TIME=slice(f"{yy}-1-1",f"{yy}-12-31")).mean(dim='TIME')
    melt = xr.where(mmelt==0,np.nan,mmelt)
    
    melt = xr.where(llon<x2,melt,np.nan)
    melt = xr.where(llat<y2,melt,np.nan)
    melt = melt.mean(dim=['LONGITUDE','LATITUDE'],skipna=True)
    MKW[y] = np.nanmean(melt)
ds.close()
    
MCDl = {}
MKWl = {}
for r,res in enumerate(ress):
    MCDl[res] = np.nan*np.zeros(len(years))
    MKWl[res] = np.nan*np.zeros(len(years))
    for y,yy in enumerate(years):
        try:
            ds = xr.open_dataset(f'../../results/CrossDots_{res}_mitgcm_{yy}_{yy}_{tpers[r]:03.0f}.nc')
            MCDl[res][y] = np.nanmean(np.where(ds.tmask==1,ds.melt,np.nan))
            melt = ds.melt.copy()
            melt = xr.where(ds.lon<x2,melt,np.nan)
            melt = xr.where(ds.lat<y2,melt,np.nan)
            MKWl[res][y] = np.nanmean(np.where(ds.tmask==1,melt,np.nan))            
            ds.close()
        except:
            pass

idx = np.argmin((MCD-np.median(MCD))**2)
print(years[idx],MCD[idx],MCDl['2.0'][idx],MCDl['1.0'][idx],MCDl['0.5'][idx])
print(years[idx],MKW[idx],MKWl['2.0'][idx],MKWl['1.0'][idx],MKWl['0.5'][idx])

In [ ]:
plt.style.use('style_paper')

mpl.rc('figure.subplot',left=.1,right=.95,top=.9,bottom=.15,wspace=.3,hspace=.3)

fig,ax = plt.subplots(2,3,figsize=(7,3.5),sharex=True)

#ax[0].plot(years,MCD,color='.5',label='MITgcm',zorder=0)
#ax[1].plot(years,MKW/MCD,color='.5',label='MITgcm',zorder=0)

#Layer

for r,res in enumerate(ress):
    ax[0,0].plot(years,MCD[res],c=cols[r],label=f'{res} km',lw=1)
    ax[0,1].plot(years,MKW[res]/MCD[res],c=cols[r],label=f'{res} km',lw=1)
    ax[1,0].plot(years,MDC[res]/MCD[res],c=cols[r],label=f'{res} km',lw=1)
    ax[1,1].plot(years,MCC[res]/MCD[res],c=cols[r],label=f'{res} km',lw=1)
    ax[1,2].plot(years,OVE[res],c=cols[r],label=f'{res} km',lw=1)

#ax[0,0].set_title('a) Crosson Dotson average',loc='left')
ax[0,0].text(.05,.9,'a)',transform=ax[0,0].transAxes)
ax[0,0].set_ylabel('Crosson-Dotson melt\n[m yr$^{-1}$]')

ax[0,1].text(.05,.9,'b)',transform=ax[0,1].transAxes)
ax[0,1].set_ylabel('Kohler West amplif.')

ax[1,0].text(.05,.9,'c)',transform=ax[1,0].transAxes)
ax[1,0].set_ylabel('Dotson Channel amplif.')

ax[1,1].text(.05,.9,'d)',transform=ax[1,1].transAxes)
ax[1,1].set_ylabel('Channel Center amplif.')

ax[1,2].text(.05,.9,'e)',transform=ax[1,2].transAxes)
ax[1,2].set_ylabel('Overturning [Sv]')



ax[0,0].fill_between([2003,2008],[8.7,8.7],[9.9,9.9],color='.8',zorder=0,label='Tuning target')
ax[0,1].fill_between([2003,2008],[3,3],[3.8,3.8],color='.8',zorder=0,label='Tuning target')
ax[1,0].fill_between([2003,2008],[1.1,1.1],[1.3,1.3],color='.8',zorder=0,label='Tuning target')
ax[1,1].fill_between([2003,2008],[2.4,2.4],[3.2,3.2],color='.8',zorder=0,label='Tuning target')
ax[1,2].fill_between([2003,2008],[.39,.39],[.61,.61],color='.8',zorder=0,label='Tuning target')


ax[0,2].set_visible(False)
ax[0,1].legend(loc='upper left', bbox_to_anchor=(1.3, 1))

ax[1,1].set_xlabel('Year')





#ax[0].set_ylim([0,16])
#ax[1].set_ylim([0,4])
plt.savefig('../../figures/draftplot_timeseries.png',dpi=450)